In [1]:
import sys
sys.version

'3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 16:05:27) [MSC v.1900 64 bit (AMD64)]'

# 平行多导线系统参数计算函数(Carson简化法计算zii 考虑集肤效应）

文件名： conductors_calc_para(r').pynb
日期： 2019.02.10
编写人： 朱明

 ## 多导线原始参数
 
 **牵引网导线架设横截面示意图**

  ![%E5%A4%8D%E7%BA%BFAT%E5%AF%BC%E7%BA%BF%E7%A9%BA%E9%97%B4%E5%88%86%E5%B8%83%E5%9B%BE.vsd.jpg](attachment:%E5%A4%8D%E7%BA%BFAT%E5%AF%BC%E7%BA%BF%E7%A9%BA%E9%97%B4%E5%88%86%E5%B8%83%E5%9B%BE.vsd.jpg)   




** 导线编号：**
 
|导线号 | 导线代码 | 导线名称 |
|:---|:---|:---|
|01|CW1|接触导线1|
|02|MW1|承力索1|
|03|PF1|正馈线1|
|04|RA1|钢轨1|
|05|RA2|钢轨2|
|06|PW1|保护线1|
|07|E1|综合地线1|
|08|CW2|接触导线2|
|09|MW2|承力索2|
|10|PF2|正馈线2|
|11|RA3|钢轨3|
|12|RA4|钢轨4|
|13|PW2|保护线2|
|14|E2|综合地线2|
  
 


** 导线架设截面坐标：**

conductors_coordinater=0.001*np.array([[0,6300],[0,7500],[-4400,8500],[-755,1000],[755,1000],[-3600,8000],[-4400,500],
                                 [5000,6300],[5000,7500],[4400+5000,8500],[4245,1000],[5755,1000],[3600+5000,8000],
                                 [4400+5000,500]],np.float64)  # 多导体坐标数组 (x,y),单位 m

** 导线计算半径：**
conductors_calc_radius=0.001*np.array([5.9,7.00,9.5,109.1,109.1,7.60,5.35,5.9,7.00,9.5,109.1,109.1,7.60,5.35])
                                    #多导体计算半径，单位 m， 计算电位系数用
** 导线等效半径：**
conductors_equivalent_radius=0.001*np.array([6.18,4.74,9.03,49.65,49.65,7.22,4.055,6.18,4.74,9.03,49.65,49.65,7.22,4.055])
                                    #多导体等效半径,单位m, 计算自电感用
** 导线直流电阻：**
conductors_resistance=np.array([0.146,0.158,0.163,0.135,0.135,0.255,0.28,0.146,0.158,0.163,0.135,0.135,0.255,0.28])
                                    #多导体直流电阻，单位 欧/km
                                    
  ** 导线电阻率：**
conductors_rho=10**-10*np.array([176.67,146.15,298.96,10405.8,10405.8,352.74,251.77,176.67,146.15,298.96,10405.8,10405.8,352.74,251.77]）计算方法：rho=RS/L（R为直流电阻，S根据导线型号查出截面半径，L=1m。单位 米）
{原始参数在软件开发前期用数组形式给出，在后阶段以数据文件的形式存储，通过读取数据生成变量。}

In [14]:
import numpy as np
import math
from scipy import linalg
from scipy import constants as C
from scipy import special

#导线定义

"""
1. 接触线（CW1）; 2.承力索（MW1）; 3.正馈线（PF1）; 4.钢轨1（RA1）;5.钢轨2（RA2）;6.保护线（PW1）;7.综合地线（E1）
8. 接触线（CW2）; 9.承力索（MW2）; 10.正馈线（PF2）; 11.钢轨3（RA3）;12.钢轨4（RA4）;13.保护线（PW2）;14.综合地线（E2）

"""
conductors_coordinater=0.001*np.array([[0,6300],[0,7500],[-4400,8500],[-755,1000],[755,1000],[-3600,8000],[-4400,500],
                                 [5000,6300],[5000,7500],[4400+5000,8500],[4245,1000],[5755,1000],[3600+5000,8000],
                                 [4400+5000,500]],np.float64)  # 多导体坐标数组 (x,y),单位 m

conductors_calc_radius=0.001*np.array([5.9,7.00,9.5,109.1,109.1,7.60,5.35,5.9,7.00,9.5,109.1,109.1,7.60,5.35])
                                    #多导体计算半径，单位 m， 计算电位系数用

conductors_equivalent_radius=0.001*np.array([4.6,4.74,9.03,49.65,49.65,7.22,4.055,4.6,4.74,9.03,49.65,49.65,7.22,4.055])
                                    #多导体等效半径,单位m, 计算自电感用


conductors_resistance=np.array([0.146,0.158,0.163,0.135,0.135,0.255,0.28,0.146,0.158,0.163,0.135,0.135,0.255,0.28])
                                    #多导体直流电阻，单位 欧/km
#conductors_rho=10**-10*np.array([159.66,243.22,462.15,50481.60,50481.60,462.72,251.77,159.66,243.22,462.15,50481.60,50481.60,462.72,251.77])、、S用πr**2的方法计算
#conductors_rho=10**-10*np.array([176.67,146.15,298.96,10405.8,10405.8,352.74,251.77,176.67,146.15,298.96,10405.8,10405.8,352.74,251.77])
conductors_rho=10**-10*np.array([55.8,54.05,28.33,2100,2100,28.33,54.05,55.8,54.05,28.33,2100,2100,28.33,54.05])
conductors_coordinater,conductors_calc_radius,conductors_equivalent_radius,conductors_resistance,conductors_rho

(array([[ 0.   ,  6.3  ],
        [ 0.   ,  7.5  ],
        [-4.4  ,  8.5  ],
        [-0.755,  1.   ],
        [ 0.755,  1.   ],
        [-3.6  ,  8.   ],
        [-4.4  ,  0.5  ],
        [ 5.   ,  6.3  ],
        [ 5.   ,  7.5  ],
        [ 9.4  ,  8.5  ],
        [ 4.245,  1.   ],
        [ 5.755,  1.   ],
        [ 8.6  ,  8.   ],
        [ 9.4  ,  0.5  ]]),
 array([0.0059, 0.007 , 0.0095, 0.1091, 0.1091, 0.0076, 0.0053, 0.0059, 0.007 , 0.0095, 0.1091, 0.1091, 0.0076, 0.0053]),
 array([0.0046, 0.0047, 0.009 , 0.0496, 0.0496, 0.0072, 0.0041, 0.0046, 0.0047, 0.009 , 0.0496, 0.0496, 0.0072, 0.0041]),
 array([0.146, 0.158, 0.163, 0.135, 0.135, 0.255, 0.28 , 0.146, 0.158, 0.163, 0.135, 0.135, 0.255, 0.28 ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

## 电位系数矩阵计算
  **函数名：  calc_potential_coefficient(c_xy,r)** 
  
 * parameters:    
       1. c_xy:  2×n维数组，多导线的坐标（x,y）,单位m;  
       2. r:     1×n维数组，导线的半径,单位 m  
 * Return:
       1. P:     n×n维数组，电位系数   
 * 功能：        计算多导体的电位系数矩阵P。 
 
  计算公式：  
  
   $$ P_{ii}=\frac{1}{2\pi\epsilon}ln\frac{2h_{i}}{r_i}=18 ×10^9ln\frac{2h_{i}}{r_i}$$
   
   $$ P_{ij}=\frac{1}{2\pi\epsilon}ln\frac{D_{ij}}{d_{ij}}=18 ×10^9ln\frac{D_{ij}}{d_{ij}}  ~~~~ (i≠j) $$
   
   
   
  ![ 导体镜像截面图](/AT_calc/images/conductors_section.png "")
  
  ![conductors_section.png](attachment:conductors_section.png)
  

In [15]:
def calc_potential_coefficient(c_xy,r):
    """ 计算电位系数矩阵P
    """
  #  epsilon0=1/(36*np.pi)*10**-9         #空气介电系数
   # c_xy=conductors_coordinater          # 多导体坐标数组 (x,y),单位 m
   # R=conductors_calc_radius             #多导体计算半径，单位 m， 计算电位系数用
    n=np.shape(c_xy)[0]
    P=np.empty((n,n),np.float64)
    for i in range(n):
        for j in range(n):
            if i==j:
                P[i,i]=18*10**+9*np.log(2*c_xy[i,1]/r[i])
            else:
                Dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]+c_xy[j,1])**2)
                dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]-c_xy[j,1])**2)
                P[i,j]=18*10**+9*np.log(Dij/dij)
    return P
#r=conductors_equivalent_radius
r=conductors_calc_radius
c_xy=conductors_coordinater
P=calc_potential_coefficient(c_xy,r)
print(P)

[[1.3800e+11 4.3962e+10 2.0588e+10 5.6780e+09 5.6780e+09 2.3569e+10 1.9196e+09 1.7953e+10 1.8880e+10 1.0741e+10 3.9241e+09 3.1031e+09 1.1586e+10 8.8458e+08]
 [4.3962e+10 1.3806e+11 2.3441e+10 4.7789e+09 4.7789e+09 2.6579e+10 1.7854e+09 1.8880e+10 2.0723e+10 1.2141e+10 3.6357e+09 3.0156e+09 1.2988e+10 9.3277e+08]
 [2.0588e+10 2.3441e+10 1.3481e+11 3.5827e+09 3.0956e+09 5.1530e+10 2.1201e+09 1.0741e+10 1.2141e+10 8.3095e+09 2.0769e+09 1.7404e+09 8.6241e+09 5.8208e+08]
 [5.6780e+09 4.7789e+09 3.5827e+09 5.2355e+10 9.1185e+09 4.0049e+09 1.2403e+09 3.1031e+09 3.0156e+09 1.7404e+09 1.3358e+09 8.1172e+08 1.8953e+09 1.7246e+08]
 [5.6780e+09 4.7789e+09 3.0956e+09 9.1185e+09 5.2355e+10 3.4724e+09 6.4720e+08 3.9241e+09 3.6357e+09 2.0769e+09 2.5558e+09 1.3358e+09 2.2881e+09 2.3690e+08]
 [2.3569e+10 2.6579e+10 5.1530e+10 4.0049e+09 3.4724e+09 1.3774e+11 2.2305e+09 1.1586e+10 1.2988e+10 8.6241e+09 2.2881e+09 1.8953e+09 9.0056e+09 6.1761e+08]
 [1.9196e+09 1.7854e+09 2.1201e+09 1.2403e+09 6.4720e+08 2

In [16]:
def calc_Y(f,c_xy,r):
    P = calc_potential_coefficient(c_xy,r)
    B=linalg.inv(P)#矩阵求逆
    Y= 1j*2*np.pi*f*B
    return Y
r=conductors_calc_radius
c_xy=conductors_coordinater
f=50
Y=calc_Y(f,c_xy,r)
print('Y矩阵(×e-12) : \n {}'.format(Y*10**12))

Y矩阵(×e-12) : 
 [[0.+2622.1434j 0. -704.0652j 0. -157.1411j 0. -145.0386j 0. -141.3895j 0. -211.9977j 0.  -24.9875j 0. -139.7631j 0. -144.8383j 0.  -48.3089j 0.  -78.5279j 0.  -46.5049j 0.  -52.6635j 0.   -5.5804j]
 [0. -704.0652j 0.+2655.1227j 0. -208.7617j 0.  -89.7028j 0.  -87.0419j 0. -269.1982j 0.  -18.899j  0. -144.8383j 0. -179.6574j 0.  -67.7054j 0.  -57.3768j 0.  -38.5989j 0.  -70.2655j 0.   -5.7884j]
 [0. -157.1411j 0. -208.7617j 0.+2772.5512j 0.  -62.8646j 0.  -40.4564j 0. -950.5885j 0.  -29.927j  0.  -48.3089j 0.  -67.7054j 0.  -42.3855j 0.  -19.5645j 0.  -13.5525j 0.  -38.7493j 0.   -2.6747j]
 [0. -145.0386j 0.  -89.7028j 0.  -62.8646j 0.+6225.377j  0.-1039.1795j 0.  -77.1413j 0.  -65.7305j 0.  -46.5049j 0.  -38.5989j 0.  -13.5525j 0.  -69.492j  0.  -28.3132j 0.  -15.3746j 0.   -2.4095j]
 [0. -141.3895j 0.  -87.0419j 0.  -40.4564j 0.-1039.1795j 0.+6235.4275j 0.  -50.9698j 0.  -20.235j  0.  -78.5279j 0.  -57.3768j 0.  -19.5645j 0. -228.6784j 0.  -69.492j  0.  -23.5343j 0.   

 
  ** 函数名: merge_potential_coefficient(P,m,k)**
 * parameters:    
       1. P:      n×n 维数组，电位系数 ;  
       2. m:      第 m 号导线（导线序号从0开始）；
       3. k:      第 k 号导线，m＜k≤n,
 * Return:
       1. P:      n-1×n-1 维数组，归并后的电位系数   
   
 * 功能：         导线k并入导线m,计算修正后的电位系数P
 
  导线k并入m公式推导：
 
 \begin{equation} 
 \begin{bmatrix}
  u_1\\
  \vdots\\
  u_m\\
  \vdots\\
  u_k\\
  \vdots\\
  u_n
  \end{bmatrix} =
  \begin{bmatrix}
  P_{11}&\dots& P_{1m}&\dots&P_{1k}&\dots&P_{1n}\\
  \vdots& &\vdots& &\vdots& &\vdots\\
  P_{m1}&\dots& P_{mm}&\dots&P_{mk}&\dots&P_{mn}\\
   \vdots& &\vdots& &\vdots& &\vdots\\
    P_{k1}&\dots& P_{km}&\dots&P_{kk}&\dots&P_{kn}\\
   \vdots& &\vdots& &\vdots& &\vdots\\ 
    P_{n1}&\dots& P_{nm}&\dots&P_{nk}&\dots&P_{nn}\\
   \end{bmatrix}
   \begin{bmatrix}
   q_1\\
   \vdots\\
  q_m\\
  \vdots\\
  q_k\\
  \vdots\\
  q_n
 \end{bmatrix}
 \end{equation}

令 $q_{m}=q_{m}+q_{k}$ 有： 

\begin{equation} 
 \begin{bmatrix}
  u_1\\
  \vdots\\
  u_m\\
  \vdots\\
  u_k\\
  \vdots\\
  u_n
  \end{bmatrix} =
  \begin{bmatrix}
  P_{11}&\dots& P_{1m}&\dots&(P_{1k}-P_{1m})&\dots&P_{1n}\\
  \vdots& &\vdots& &\vdots& &\vdots\\
  P_{m1}&\dots& P_{mm}&\dots&(P_{mk}-P_{mm})&\dots&P_{mn}\\
  \vdots& &\vdots& &\vdots& &\vdots\\
   P_{k1}&\dots& P_{km}&\dots&(P_{kk}-P_{km})&\dots&P_{kn}\\
  \vdots& &\vdots& &\vdots& &\vdots\\ 
   P_{n1}&\dots& P_{nm}&\dots&(P_{nk}-P_{nm})&\dots&P_{nn}\\
  \end{bmatrix}
  \begin{bmatrix}
   q_1\\
   \vdots\\
  q_m+q_k\\
  \vdots\\
  q_k\\
  \vdots\\
  q_n
 \end{bmatrix}
\end{equation}   

P矩阵第k列参数发生变化，修正公式为：$$P_{ik}=P_{ik}-P_{im}$ ( i=0,1,...,n), q_m=q_m+q_k$$.

方程中，$u_{k}-u_{m}=0$,得方程

 \begin{equation*} 
 \begin{bmatrix}
  u_1\\
  \vdots\\
  u_m\\
  \vdots\\
  0\\
  \vdots\\
  u_n
  \end{bmatrix} =
  \begin{bmatrix}
  P_{11}&\dots& P_{1m}&\dots&P_{1k}&\dots&P_{1n}\\
  \vdots& &\vdots& &\vdots& &\vdots\\
  P_{m1}&\dots& P_{mm}&\dots&P_{mk}&\dots&P_{mn}\\
   \vdots& &\vdots& &\vdots& &\vdots\\
    P_{k1}- P_{m1}&\dots& P_{km}-P_{mm}&\dots&P_{kk}-P_{mk}&\dots&P_{kn}-P_{mn}\\
   \vdots& &\vdots& &\vdots& &\vdots\\ 
    P_{n1}&\dots& P_{nm}&\dots&P_{nk}&\dots&P_{nn}\\
   \end{bmatrix}
   \begin{bmatrix}
   q_1\\
   \vdots\\
  q_m\\
  \vdots\\
  q_k\\
  \vdots\\
  q_n
 \end{bmatrix}
\end{equation*}

第k行等式有:
$$ 0= (P_{k1}- P_{m1})q_1+\dots+（P_{km}-P_{mm})q_m+\dots+(P_{kk}-P_{mk})q_k+\dots+(P_{kn}-P_{mn})q_n$$

可以求得：
$$q_k=-\frac{(P_{k1}- P_{m1})}{(P_{kk}-P_{mk})}q_1-\dots-\frac{(P_{km}- P_{mm})}{(P_{kk}-P_{mk})}q_m\dots-\frac{(P_{kn}- P_{mn})}{(P_{kk}-P_{mk})}q_n$$

令$K_j=\frac{P_{kj}- P_{mj}}{P_{kk}-P_{mk}}$ (j≠k)，有
$$q_k=-K_1q_1-\dots-K_m q_m-\dots-K_n q_n$$

导线电压$u_i$写成方程有：
$$u_i=P_{i1}q_1+\dots+P_{im}q_m+\dots+P_{ik}q_k+\dots+P_{in}q_n$$

把$q_k$代入方程中，得到
$$u_i=（P_{i1}-P_{ik}K_1)q_1+\dots+(P_{im}-P_{ik}K_m)q_m+\dots+(P_{in}-P_{ik}K_n)q_n$$

取(i,j=0,1,...,n; i≠k;j≠k),可以得到合并矩阵元素$P_{ij}$.
 $$P_{ij}=P_{ij}-P_{ik}K_j=P_{ij}-P_{ik}\frac{P_{kj}- P_{mj}}{P_{kk}-P_{mk}}$$

总之，导线k合并到导线m，P参数的修正步骤归纳如下：

（1）$P_{ik}=P_{ik}-P_{im}$ ( i=0,1,...,n)

（2）$P_{ij}=P_{ij}-P_{ik}K_j=P_{ij}-P_{ik}\frac{P_{kj}- P_{mj}}{P_{kk}-P_{mk}}$ （i,j=0,1,...,n; i≠k;j≠k）

（3）除P中的第k行第k列，P降n-1阶，为合并后n-1根导线的电位系数矩阵。

In [17]:
def merge_potential_coefficient(P,m,k):
    n=np.shape(P)[0]
    for i in range(n):
        P[i,k]=P[i,k]-P[i,m]
         
    for i in range(n):
        for j in range(n):
            if i!=k or j!=k:
                P[i,j]=P[i,j]-(P[k,j]-P[m,j])/(P[k,k]-P[m,k])*P[i,k]
                        
          
    E=np.empty((n-1,n-1),np.float64)
    for i in range(n):
        for j in range(n):
            if i<k:
                if j<k:
                    E[i,j]=P[i,j]
                if j>k:
                    E[i,j-1]=P[i,j]
            if i>k:
                if j<k:
                    E[i-1,j]=P[i,j]
                if j>k:
                    E[i-1,j-1]=P[i,j]
    
    return E

#测试
P=calc_potential_coefficient(conductors_coordinater,conductors_calc_radius)
# P=merge_potential_coefficient(P,0,1) 
# P=merge_potential_coefficient(P,2,3)
# P=merge_potential_coefficient(P,2,3)
# P=merge_potential_coefficient(P,2,3)
# P=merge_potential_coefficient(P,3,4)
# P=merge_potential_coefficient(P,5,6)
# P=merge_potential_coefficient(P,5,6)
# P=merge_potential_coefficient(P,5,6)
P=merge_potential_coefficient(P,0,1)
P=merge_potential_coefficient(P,6,7) 
P=merge_potential_coefficient(P,2,3)
P=merge_potential_coefficient(P,7,8) 
P=merge_potential_coefficient(P,2,3)
P=merge_potential_coefficient(P,6,7) 
P=merge_potential_coefficient(P,2,3)
P=merge_potential_coefficient(P,5,6) 
np.set_printoptions(precision=3,linewidth=214,suppress=True)
print('P矩阵 e+10 : \n {}'.format(P*10**-10))

P矩阵 e+10 : 
 [[ 8.807  1.426  0.715  1.729  0.802  0.402]
 [ 1.426 11.928  0.935  1.056  0.611  0.252]
 [ 0.715  0.935  2.095  0.291  0.142  0.099]
 [ 1.729  1.056  0.291  8.863  1.472  0.725]
 [ 0.802  0.611  0.142  1.472 11.97   0.947]
 [ 0.402  0.252  0.099  0.725  0.947  2.068]]


 
   
   AT牵引网络由14根导线逐步合并成6根导线：
   
  （1） 1. 接触线（CW1）+ 2.承力索（MW1）
  
  （2） 3.正馈线（PF1）
  
  （3） 4.钢轨1（RA1）+5.钢轨2（RA2）+6.保护线（PW1）+7.综合地线（E1）
  
  （4） 8. 接触线（CW2）+ 9.承力索（MW2）
  
  （5） 10.正馈线（PF2）
  
  （6） 11.钢轨3（RA3）+12.钢轨4（RA4）+13.保护线（PW2）+14.综合地线（E2）
  
   
##  电容系数矩阵计算
  函数名： calc_B(P)
  

In [18]:
def calc_B():
    """ 计算电容系数矩阵
    """
    P=calc_potential_coefficient(conductors_coordinater,conductors_calc_radius)
    P=merge_potential_coefficient(P,0,1) 
    P=merge_potential_coefficient(P,2,3)
    P=merge_potential_coefficient(P,2,3)
    P=merge_potential_coefficient(P,2,3)
    P=merge_potential_coefficient(P,3,4)
    P=merge_potential_coefficient(P,5,6)
    P=merge_potential_coefficient(P,5,6)
    P=merge_potential_coefficient(P,5,6)
    B=linalg.inv(P)
    return B
B=calc_B()
np.set_printoptions(precision=3,linewidth=214,suppress=True)              
          
print('B矩阵(×e-12) : \n {}'.format(B*10**12))

B矩阵(×e-12) : 
 [[12.273 -0.979 -3.388 -2.002 -0.389 -1.226]
 [-0.979  8.855 -3.489 -0.68  -0.232 -0.375]
 [-3.388 -3.489 50.569 -0.482 -0.043 -1.149]
 [-2.002 -0.68  -0.482 12.218 -1.067 -3.298]
 [-0.389 -0.232 -0.043 -1.067  8.805 -3.553]
 [-1.226 -0.375 -1.149 -3.298 -3.553 51.488]]


##  理想导体时电感L矩阵计算  

**函数名：calc_L(c_xy,r)**  
* Parameters:
    1. c_xy:      2×n维数组，多导线的坐标（x,y）,单位（m）;  
    2. r:         1×n维数组，导线的半径,单位(m)  
    
* Return:
    1. L:        n×n维数组，多导体和大地为理想导体时的自感$L_{ii}$和互感$L_{ij}$  
    
功能：    计算理想导体时的自感$L_{ii}$和互感$L_{ij}$  

**计算公式：**

导线回路电感 $$L_{ii}=\frac{\mu_0}{2\pi}ln\frac{2h_i}{r_i}=2×10^{-4}ln\frac{2h_i}{r_i}$$
导线回路互感 $$L_{ij}=\frac{\mu_0}{2\pi}ln\frac{D_{ij}}{d_{ij}}=2×10^{-4}ln\frac{D_{ij}}{d_{ij}}$$


In [19]:
def calc_L(c_xy,r):
    n=np.shape(c_xy)[0]
    L=np.empty((n,n),np.float64)
    for i in range(n):           #计算导线外自感和互电感
        for j in range(n):
            if i==j:
                L[i,i]=2*10**-7*np.log(2*c_xy[i,1]/r[i])
            else:
                Dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]+c_xy[j,1])**2) 
                dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]-c_xy[j,1])**2)
                L[i,j]=2*10**-7*np.log(Dij/dij) 
    return L

# 函数测试
f=50
c_xy=conductors_coordinater
r=conductors_calc_radius
L=calc_L(c_xy,r)
np.set_printoptions(precision=4,linewidth=214,suppress=True) 
print('L矩阵(×e-3) : \n {}'.format(L*10**3))


L矩阵(×e-3) : 
 [[0.0015 0.0005 0.0002 0.0001 0.0001 0.0003 0.     0.0002 0.0002 0.0001 0.     0.     0.0001 0.    ]
 [0.0005 0.0015 0.0003 0.0001 0.0001 0.0003 0.     0.0002 0.0002 0.0001 0.     0.     0.0001 0.    ]
 [0.0002 0.0003 0.0015 0.     0.     0.0006 0.     0.0001 0.0001 0.0001 0.     0.     0.0001 0.    ]
 [0.0001 0.0001 0.     0.0006 0.0001 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.0001 0.0001 0.     0.0001 0.0006 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.0003 0.0003 0.0006 0.     0.     0.0015 0.     0.0001 0.0001 0.0001 0.     0.     0.0001 0.    ]
 [0.     0.     0.     0.     0.     0.     0.001  0.     0.     0.     0.     0.     0.     0.    ]
 [0.0002 0.0002 0.0001 0.     0.     0.0001 0.     0.0015 0.0005 0.0002 0.0001 0.0001 0.0003 0.    ]
 [0.0002 0.0002 0.0001 0.     0.     0.0001 0.     0.0005 0.0015 0.0003 0.0001 0.0001 0.0003 0.    ]
 [0.0001 0.0001 0.0001 0.     0.     0.0001 0.     0.0002 0.0003 0.0015 0.   

## 计算导线内电阻和内电感
**函数名： calc_Zc(f,R,rho)**
* Parameters:
    1. f:       频率（Hz）
    2. R:      导线半径（m）(等效半径） 
    3. rhoi     导线电阻率（Ω/m）
* Return:
    1. Rc:      导线交流电阻（Ω/m）
    2. Xc:      导线交流内电感（H/m）  
    
 功能：计算导线的交流电阻Rc和内电感Lc
 
 实心圆柱体单位长度交流内阻抗为  
 $$ Z_c=\frac{jmρJ_0(nr)}{2\pi J_1(nr)}=\frac{jmρ}{2\pi r}\frac{ber(mr)+jbei(mr)}{ber'(mr)+jbei'(mr)}$$
 式中$n=jm\sqrt j  ,m=\sqrt\frac{ωμ}{ρ}$  
 
 导线内阻一般用上述基于贝塞尔函数的公式计算，如果导线是钢轨，由于是铁磁材质，需要通过用有限元软件计算给出。

In [20]:
def calc_Zc(f,R,mu,rho):
    #n=np.shape(rho)[0]
    #m=np.empty(n,np.float64)
    #Zc=np.empty(n,np.complex128)
    #for i in range(n):
    m=np.sqrt(2*np.pi*f*mu/rho)
    A=1j*m*rho/(2*np.pi*R)
    B=special.ber(m*R)+1j*special.bei(m*R)
    C=special.berp(m*R)+1j*special.beip(m*R)
    Zc=A*B/C
    #print(m) 
    return Zc

#测试
f=50
R=conductors_equivalent_radius          # 单位 m
mu=4*np.pi*10**-7     #
rho=conductors_rho
Zc=calc_Zc(f,R,mu,rho)
print(Zc*10**3)
print(Zc.tolist())

[0.0849+0.0156j 0.0776+0.0156j 0.016 +0.0124j 0.0299+0.0149j 0.0299+0.0149j 0.0212+0.014j  0.1054+0.0156j 0.0849+0.0156j 0.0776+0.0156j 0.016 +0.0124j 0.0299+0.0149j 0.0299+0.0149j 0.0212+0.014j  0.1054+0.0156j]
[(8.491071305089507e-05+1.561720197393702e-05j), (7.76375343385756e-05+1.55991243450224e-05j), (1.5983081526977534e-05+1.2353873135476221e-05j), (2.9902426681558245e-05+1.4906918230284273e-05j), (2.9902426681558245e-05+1.4906918230284273e-05j), (2.121215356489907e-05+1.396266264474747e-05j), (0.00010541326375920289+1.564934106454878e-05j), (8.491071305089507e-05+1.561720197393702e-05j), (7.76375343385756e-05+1.55991243450224e-05j), (1.5983081526977534e-05+1.2353873135476221e-05j), (2.9902426681558245e-05+1.4906918230284273e-05j), (2.9902426681558245e-05+1.4906918230284273e-05j), (2.121215356489907e-05+1.396266264474747e-05j), (0.00010541326375920289+1.564934106454878e-05j)]


## 计算大地与导线回路电阻和电感
**函数名：calc_Zgm(f,c_xy，rou)**  
*  Parameters:
    1. f:       频率（Hz）
    2. c_xy:    2×n维数组，多导线的坐标（x,y）,单位（m）; 
    3. rou:     大地电阻率（Ω/m）
* Return:
    1. Rgm:      n×n维数组,大地与导线回路电阻（Ω/km）
    2. Xgm:      n×n维数组,大地与导线回路感抗（Ω/km）  
    
 功能：计算导线与大地的互阻抗,需要调用函数calc_Rg()，calc_Xg()



In [21]:
def calc_Zgm(f,c_xy,rou):
    n=np.shape(c_xy)[0]
    Rgm=np.empty((n,n),np.float64)
    Xgm=np.empty((n,n),np.float64)
    for i in range(n):          
        for j in range(n):
            Dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]+c_xy[j,1])**2)
            xij=np.abs(c_xy[i,0]-c_xy[j,0])
            theta=np.arcsin(xij/Dij)
            k=4*np.pi*np.sqrt(5)*10**-4*Dij*np.sqrt(f/rou) 
            Rgm[i,j]=calc_Rg(f,k,theta)
            Xgm[i,j]=calc_Xg(f,k,theta)
                
    return Rgm,Xgm



def calc_Rg(f,k,theta):
       
    b1=np.sqrt(2)/6
    b2=1/16
    b3=b1/(3*5)
    b4=b2/(4*6)
    b5=-b3/(5*7)
    b6=-b4/(6*8)
    b7=-b5/(7*9)
    b8=-b6/(8*10)
    
    c2=1.3659315
    c4=c2+1/4+1/6
    c6=c4+1/6+1/8
    
    d4=np.pi/4*b4
    d6=np.pi/4*b6
    d8=np.pi/4*b8
    
    if k<5.1:
        Rg=np.pi/8
        -b1*k*np.cos(theta)
        +b1*k*np.cos(theta)
        +b2*((c2-np.log(k))*k**2*np.cos(2*theta)+theta*k**2*np.sin(2*theta))
        +b3*k**3*np.cos(3*theta)
        -d4*k**4*np.cos(4*theta)
        -b5*k**5*np.cos(5*theta)
        +b6*((c6-np.log(k))*k**2*np.cos(6*theta)+theta*k**6*np.sin(6*theta))
        +b7*np.cos(7*theta)
        -d8*k**8*np.cos(8*theta)
    else:
        Rg=np.cos(theta)/k
        -np.sqrt(2)*np.cos(2*theta)/k**2
        +np.cos(3*theta)/k**3
        +3*np.cos(5*theta)/k**5
        -45*np.cos(7*theta)
        Rg=Rg/sqrt(2)
      
    Rg=4*2*np.pi*f*10**-4*Rg
    return Rg
    
def calc_Xg(f,k,theta):
                
    b1=np.sqrt(2)/6
    b2=1/16
    b3=b1/(3*5)
    b4=b2/(4*6)
    b5=-b3/(5*7)
    b6=-b4/(6*8)
    b7=-b5/(7*9)
    b8=-b6/(8*10)
    
    c2=1.3659315
    c4=c2+1/4+1/6
    c6=c4+1/6+1/8
    c8=c4+1/8+1/10
    
    d2=np.pi/4*b2
    d4=np.pi/4*b4
    d6=np.pi/4*b6
    d8=np.pi/4*b8
    
    if k<5.1:
        Xg=0.5*(0.6159315-np.log(k))
        +b1*k*np.cos(theta)
        -d2*k*k*np.cos(2*theta)
        +b3*k**3*np.cos(3*theta)
        -b4*((c4-np.log(k))*k**4*np.cos(4*theta)+theta*k**4*np.sin(4*theta))
        +b5*k**5*np.cos(5*theta)
        -d6*k**6*np.cos(6*theta)
        +b7*k**7*np.cos(7*theta)
        -b8*((c8-np.log(k))*k**8*np.cos(8*theta)+theta*k**8*np.sin(8*theta))
    else:
        Xg=np.cos(theta)/k
        -np.cos(3*theta)/k**3
        +3*np.cos(5*theta)-45*np.cos(7*theta)
        Xg=Xg/sqrt(2)
    
    Xg=4*2*np.pi*f*10**-4*Xg
    return Xg

# 测试
f=50
rou=10**6
c_xy=conductors_coordinater
Rgm,Xgm=calc_Zgm(f,c_xy,rou)
np.set_printoptions(precision=4,linewidth=214,suppress=True)
print('Rgm 矩阵 : \n {}'.format(Rgm))
print('Xgm 矩阵 : \n {}'.format(Xgm))


Rgm 矩阵 : 
 [[0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0

In [22]:
f=2000
rou=100
c_xy=conductors_coordinater
n=np.shape(c_xy)[0]
for i in range(n):           #计算导线与大地回路电阻和电感
        for j in range(n):
            Dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]+c_xy[j,1])**2)
            xij=np.abs(c_xy[i,0]-c_xy[j,0])
            theta=np.arcsin(xij/Dij)
            k=4*np.pi*np.sqrt(5)*10**-4*Dij*np.sqrt(f/rou)
            print(k)
            
            
 
            
            

0.1583362697409256
0.1734159144781566
0.1940273666973689
0.09222382683734463
0.09222382683734463
0.18530603776528737
0.10177984466181597
0.17034733950345948
0.1844476108675064
0.2203239938960755
0.10611704757157861
0.1168132022016855
0.20969277891175298
0.145791563672889
0.1734159144781566
0.1884955592153876
0.20852603742972592
0.10723468388370899
0.10723468388370899
0.19996329784837363
0.11473309708186626
0.1844476108675064
0.19869176531592203
0.23319337800406367
0.11939376439506019
0.12899367193675174
0.22275110410918417
0.15511198156428663
0.1940273666973689
0.20852603742972592
0.21362830044410597
0.1278661555227996
0.13582382184538755
0.20758868350710358
0.11309733552923257
0.2203239938960755
0.23319337800406367
0.2751547385471174
0.1614111172606516
0.17474667965698754
0.2639685721287496
0.20703643809268732
0.09222382683734463
0.10723468388370899
0.1278661555227996
0.025132741228718346
0.0314914852236045
0.11861350892715487
0.049531310617093
0.1168132022016855
0.12899367193675174
0

## 导线简化计算阻抗
** calc_z(f,c_xy,r,re,rou)** 
* Parameters:
    1. f:       频率（Hz）
    2. c_xy:    2×n维数组，多导线的坐标（x,y）,单位（m）; 
    3. r:       1×n维数组导线，等效半径（m）
    4. re       1×n维数组，导线电阻
    5. rou      大地电阻率（Ω/m）
* Return:
    1. R:      导线交流电阻（Ω/m）
    2. X:      导线交流内电感（亨/km） 
    

In [23]:
def calc_aRaL(f,r):
    #a=np.shape(r)[0]
  
    #aR=np.empty((a,b),np.float64)
    #aL=np.empty((a,b),np.float64)
    #m=np.empty((a,b),np.float64)
    #for i in range(a):
    m=np.sqrt(2*np.pi*f*mu/rho)
    A1=m*r/2
    A2=4/(m*r)
    B1=special.ber(m*r)
    B2=special.bei(m*r)
    C1=special.berp(m*r)
    C2=special.beip(m*r)
    aR=A1*((B1*C2-C1*B2)/(C1**2+C2**2))
    aL=A2*((B2*C2+C1*B1)/(C1**2+C2**2))
    return aR,aL
f=50
r=conductors_calc_radius
aR,aL=calc_aRaL(f,r)
print(aR)
print(aL)

[1.0308 1.0634 1.509  1.9463 1.9463 1.2672 1.0224 1.0308 1.0634 1.509  1.9463 1.9463 1.2672 1.0224]
[0.9846 0.9685 0.758  0.5868 0.5868 0.8692 0.9888 0.9846 0.9685 0.758  0.5868 0.5868 0.8692 0.9888]


In [45]:
def calc_z(f,c_xy,r,Re,rou):
    n=np.shape(c_xy)[0]
    R=np.empty((n,n),np.float64)
    X=np.empty((n,n),np.float64)
    z=np.empty((n,n),np.complex128)
    aR,aL=calc_aRaL(f,r)
    rp=(math.e*r)**(-aL/4)
    Rg=np.pi**2*f*10**-4
    Dg=660*np.sqrt(rou/f)
    for i in range(n):           #
        for j in range(n):
            if i==j:
                R[i,j]=Rg+Re[i]
                X[i,j]=2*2*np.pi*f*10**-4*np.log(Dg/rp[i])#rp为几何半径
            else:
                dij=np.sqrt((c_xy[i,0]-c_xy[j,0])**2+(c_xy[i,1]-c_xy[j,1])**2)
                R[i,j]=Rg
                X[i,j]=2*2*np.pi*f*10**-4*np.log(Dg/dij)
    z=R+1j*X        
    return R,X,z
# 测试该函数
f=50
c_xy=conductors_coordinater
#r=conductors_equivalent_radius
rd=conductors_resistance
Re=rd*aR
rou=100
R,X,z=calc_z(f,c_xy,r,Re,rou)
np.set_printoptions(precision=4,linewidth=214,suppress=True)
print('R 矩阵 : \n {}'.format(R))
print('X 矩阵 : \n {}'.format(X))
print('z 矩阵 : \n {}'.format(z))

R 矩阵 : 
 [[0.1998 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.2174 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.2953 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.3121 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.3121 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.3725 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.3356 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.1998 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.2174 0.0493 0.0493 0.0493 0.0493 0.0493]
 [0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.0493 0.2953 0.0493 0.0

# 5 阻抗矩阵合并

** 函数名：merge_z(z,i,j)**
* parameters:    
       1. z:      n×n 维数组，阻抗矩阵 ;  
       2. m:      第 m 号导线（导线序号从0开始）；
       3. k:      第 k 号导线，m＜k≤n,
 * Return:
       1. z:      n-1×n-1 维数组，归并后的电位系数  
       
功能： 导线j并入导线i, i＜j≤n, n为z的阶，输出合并阻抗矩阵z,z降一阶




   AT牵引网络由14根导线逐步合并成6根导线：
   
  （1） 1. 接触线（CW1）+ 2.承力索（MW1）
  
  （2） 3.正馈线（PF1）
  
  （3） 4.钢轨1（RA1）+5.钢轨2（RA2）+6.保护线（PW1）+7.综合地线（E1）
  
  （4） 8. 接触线（CW2）+ 9.承力索（MW2）
  
  （5） 10.正馈线（PF2）
  
  （6） 11.钢轨3（RA3）+12.钢轨4（RA4）+13.保护线（PW2）+14.综合地线（E2） 
 

def merge_z(z,m,k):
    P=z
    n=np.shape(P)[0]
    for i in range(n):
        P[i,k]=P[i,k]-P[i,m]
         
    for i in range(n):
        for j in range(n):
            if i!=k or j!=k:
                P[i,j]=P[i,j]-(P[k,j]-P[m,j])/(P[k,k]-P[m,k])*P[i,k]
                               
            
    E=np.empty((n-1,n-1),np.complex128)
    for i in range(n):
        for j in range(n):
            if i<k:
                if j<k:
                    E[i,j]=P[i,j]
                if j>k:
                    E[i,j-1]=P[i,j]
            if i>k:
                if j<k:
                    E[i-1,j]=P[i,j]
                if j>k:
                    E[i-1,j-1]=P[i,j]
    
    return E

# 测试
#f=50
c_xy=conductors_coordinater
re=conductors_resistance
rou=100
R,X,z=calc_z(f,c_xy,r,re,rou)

z=merge_z(z,0,1) 
z=merge_z(z,2,3)
z=merge_z(z,2,3)
z=merge_z(z,2,3)
z=merge_z(z,3,4)
z=merge_z(z,5,6)
z=merge_z(z,5,6)
z=merge_z(z,5,6)
Z=np.abs(z)
np.set_printoptions(precision=4,linewidth=214,suppress=True)              
print('阻抗矩阵z （Ω/km）: \n {}'.format(z))
print('阻抗矩阵Z （Ω/km）: \n {}'.format(Z))

# '''阻抗归并
# 归并1：PW2保护线2，RR2钢轨2右，RL2钢轨2左，CGW2综合地线2，四根导线归并为1根
# 第一步：行归并'''
Pm1=z.copy()
Pm1[:,13]=Pm1[:,13]-Pm1[:,10]
Pm1[:,12]=Pm1[:,12]-Pm1[:,10]
Pm1[:,11]=Pm1[:,11]-Pm1[:,10]
# '''第二步：列归并'''
Pm1[13,:]=Pm1[13,:]-Pm1[10,:]
Pm1[12,:]=Pm1[12,:]-Pm1[10,:]
Pm1[11,:]=Pm1[11,:]-Pm1[10,:]
# '''分块'''
PLL1=np.mat(Pm1[0:11,0:11])
PLT1=np.mat(Pm1[0:11,11:14])
PTL1=np.mat(Pm1[11:14,0:11])
PTT1=np.mat(Pm1[11:14,11:14])
# print(np.shape(PTT1))
# print(np.shape(PTL1))
# print(np.shape(PLT1))
# print(np.shape(PLL1))
#p11 = np.linalg.inv(PTT1)
#print('P11:',P11.shape,P11.type)
#print('PLT1:',PLT1.shape)
# p22 = PLT1*p11
#p22 = PLT1*p11
#print('p22:',p22.shape)
Pm2=PLL1-PLT1*np.linalg.inv(PTT1)*PTL1

# '''归并2：MW2承力索2，CW2接触线2，两根导线归并为1根
# 第一步'''
Pm2[:,8]=Pm2[:,8]-Pm2[:,7]
# '''第二步'''
Pm2[8,:]=Pm2[8,:]-Pm2[7,:]
# '''分块'''
PLL2u=np.mat(np.hstack((Pm2[0:8,0:8],Pm2[0:8,9:11])))
PLL2d=np.mat(np.hstack((Pm2[9:11,0:8],Pm2[9:11,9:11])))
PLL2=np.mat(np.vstack((PLL2u,PLL2d)))
#PLT2=Pm2[[0:7,9:10],8]
PLT2=np.mat(np.vstack((Pm2[0:8,8],Pm2[9:11,8])))
#PTL2=Pm2[8,[0:7,9:10]]
PTL2=np.mat(np.hstack((Pm2[8,0:8],Pm2[8,9:11])))
PTT2=np.mat(Pm2[8,8])
Pm3=PLL2-PLT2*np.linalg.inv(PTT2)*PTL2#第二次合并完成（10×10）

# '''归并3：PW1保护线1，RR1钢轨1右，RL1钢轨1左，CGW1综合地线1，四根导线归并为1根
# 第一步'''
Pm3[:,4]=Pm3[:,4]-Pm3[:,3]
Pm3[:,5]=Pm3[:,5]-Pm3[:,3]
Pm3[:,6]=Pm3[:,6]-Pm3[:,3]
#第二步
Pm3[4,:]=Pm3[4,:]-Pm3[3,:]
Pm3[5,:]=Pm3[5,:]-Pm3[3,:]
Pm3[6,:]=Pm3[6,:]-Pm3[3,:]
#分块
PLL3u=np.mat(np.hstack((Pm3[0:4,0:4],Pm3[0:4,7:10])))
PLL3d=np.mat(np.hstack((Pm3[7:10,0:4],Pm3[7:10,7:10])))
PLL3=np.mat(np.vstack((PLL3u,PLL3d)))
PLT3=np.mat(np.vstack((Pm3[0:4,4:7],Pm3[7:10,4:7])))
PTL3=np.mat(np.hstack((Pm3[4:7,0:4],Pm3[4:7,7:10])))
PTT3=np.mat(Pm3[4:7,4:7])
Pm4=PLL3-PLT3*np.linalg.inv(PTT3)*PTL3#第三次合并完成（7×7）

# '''归并4：MW1承力索1，CW1接触线1，两根导线归并为1根
# 第一步'''
Pm4[:,1]=Pm4[:,1]-Pm4[:,0]
# '''第二步'''
Pm4[1,:]=Pm4[1,:]-Pm4[0,:]
type(Pm4[0,2:7])
# '''分块'''
PLL4u=np.mat(np.hstack((np.mat(Pm4[0,0]),Pm4[0,2:7])))

PLL4d=np.mat(np.hstack((Pm4[2:7,0],Pm4[2:7,2:7])))
PLL4=np.mat(np.vstack((PLL4u,PLL4d)))
PLT4=np.mat(np.vstack((np.mat(Pm4[0,1]),Pm4[2:7,1])))
PTL4=np.mat(np.hstack((np.mat(Pm4[1,0]),Pm4[1,2:7])))
PTT4=np.mat(Pm4[1,1])
Pm=PLL4-PLT4*np.linalg.inv(PTT4)*PTL4#第二次合并完成（6×6）
z=Pm
Z=np.abs(z)
np.set_printoptions(precision=4,linewidth=214,suppress=True)              
print('阻抗矩阵z （Ω/km）: \n {}'.format(z))
print('阻抗矩阵Z （Ω/km）: \n {}'.format(Z))

In [46]:
def merge(z,m,k):
    n = np.shape(z)[0]
    for i in range(n):
        z[i,k]=z[i,k]-z[i,m]
    for j in range(n):
        z[k,j]=z[k,j]-z[m,j]
    
    zLL = np.empty((n-1,n-1),np.complex128)
    for i in range(n):
        for j in range(n):
            if i <k:
                if j<k:
                    zLL[i,j]=z[i,j]
                if j>k:
                    zLL[i,j-1]=z[i,j]
            if i>k:
                if j<k:
                    zLL[i-1,j]=z[i,j]
                if j>k:
                    zLL[i-1,j-1]=z[i,j]
    zLL = np.mat(zLL)
    
    zLT = np.empty((n-1,1),np.complex128)
    for i in range(n):
        if i<k:
            zLT[i,0]=z[i,k]
        if i>k:
            zLT[i-1,0]=z[i,k]
    zLT = np.mat(zLT)
    
    zTL = np.empty((1,n-1),np.complex128)
    for j in range(n):
        if j<k:
            zTL[0,j] = z[k,j]
        if j>k:
            zTL[0,j-1]=z[k,j]
    zTL = np.mat(zTL)
    
    zTT=np.empty((0,0),np.complex128)
    zTT=z[k,k]
    zTT=np.mat(zTT)
    
    E = zLL-zLT*np.linalg.inv(zTT)*zTL
    return E

In [47]:
# 测试
#f=50
c_xy=conductors_coordinater
re=conductors_resistance
rou=100
R,X,z=calc_z(f,c_xy,r,re,rou)

z=merge(z,0,1) 
z=merge(z,2,3)
z=merge(z,2,3)
z=merge(z,2,3)
z=merge(z,3,4)
z=merge(z,5,6)
z=merge(z,5,6)
z=merge(z,5,6)
Z=np.abs(z)
np.set_printoptions(precision=4,linewidth=214,suppress=True)              
print('阻抗矩阵z （Ω/km）: \n {}'.format(z))

阻抗矩阵z （Ω/km）: 
 [[0.1306+0.3913j 0.0624+0.3276j 0.0494+0.3209j 0.0513+0.3275j 0.0489+0.2889j 0.0512+0.2957j]
 [0.0624+0.3276j 0.2621+0.3623j 0.0343+0.3312j 0.0489+0.2889j 0.044 +0.2641j 0.0429+0.2774j]
 [0.0494+0.3209j 0.0343+0.3312j 0.1101+0.3497j 0.0512+0.2957j 0.0429+0.2774j 0.062 +0.2916j]
 [0.0513+0.3275j 0.0489+0.2889j 0.0512+0.2957j 0.1306+0.3913j 0.0624+0.3276j 0.0494+0.3209j]
 [0.0489+0.2889j 0.044 +0.2641j 0.0429+0.2774j 0.0624+0.3276j 0.2621+0.3623j 0.0343+0.3312j]
 [0.0512+0.2957j 0.0429+0.2774j 0.062 +0.2916j 0.0494+0.3209j 0.0343+0.3312j 0.1101+0.3497j]]
